## Importando e intalando os requisitos
 

In [8]:
### Importando bibliotecas 
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt 
import numpy as np 

In [9]:
# escalonamento das imagens - GPU
GPUS = tf.config.experimental.list_physical_devices('GPU')
for GPU in GPUS:
    tf.config.experimental.set_memory_growth(GPU, True)

## Carregando modelo

In [10]:
# Carregando o modelo
Modelo = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
# Extraindo o modelo - Padrão
Movenet = Modelo.signatures['serving_default']

## Modelando - Estrutura de detecção

In [16]:
## escolhendo Webcam

Cap = cv2.VideoCapture(0)

while Cap.isOpened():
   ret, frame = Cap.read()
   ## Redimensionando a imagem 
   imagem = frame.copy()
   imagem = tf.image.resize_with_pad(tf.expand_dims(imagem, axis=0), 192, 256)
   Entrada_imagem = tf.cast(imagem, dtype=tf.int32)

   ### detectando seleção
   Resultados = Movenet(Entrada_imagem)
   Pontos_chave = Resultados['output_0'].numpy()[:, :, :51].reshape((6,17,3))

   ## Renderizar pontos-chave
   loop_through_people(frame, Pontos_chave, edges, 0.30)

   

   cv2.imshow('Teste - Movenet Multipose', frame)
    
   if cv2.waitKey(10) & 0xFF==ord('q'):
       break 
Cap.release() 
cv2.destroyAllWindows()   



KeyboardInterrupt: 

: 

In [55]:
frame.shape

(1080, 1920, 3)

In [11]:
### Função para percorrer cada pessoa detectada e renderizar
def loop_through_people (frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

## Desenvolvendo pontos chave

In [12]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)



## Desenhando as bordas

In [13]:
edges = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [14]:
##Segmentos de conexão dos pontos

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)